In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from tqdm import tqdm

%matplotlib inline

In [3]:
os.chdir('/Users/micheleclaibourn/Box Sync/mpc/dataForDemocracy/summer-sandbox/eastern_shore_collection')
%pwd

'/Users/micheleclaibourn/Box Sync/mpc/dataForDemocracy/summer-sandbox/eastern_shore_collection'

In [7]:
# read in cville spatial data
eastshore_sp = {}
spatial_units = ['counties', 'tracts', 'blkgps', 'blocks']
path_to_data = '/data/shape/'
for su in spatial_units:
    eastshore_sp[su] = gpd.read_file(os.getcwd() + path_to_data + 'eastshore_{su}.shp'.format(su = su))

# create GEOID in blocks (is GEOID10)    
eastshore_sp['blocks']['BLKGRPCE'] = eastshore_sp['blocks'].GEOID10.apply(lambda x: str(x)[11])
eastshore_sp['blocks']['GEOID'] = eastshore_sp['blocks'].GEOID10

In [8]:
# downloaded files manually ----
# from: https://msc.fema.gov/portal/advanceSearch#searchresultsanchor
# add to dataraw folder

accomack_data_path = "./dataraw/nfhl/accomack/51001C_20211026"
northampton_data_path = "./dataraw/nfhl/northampton/51131C_20211026"

eastshore_data_paths = [
    accomack_data_path,
    northampton_data_path
]

In [9]:
def read_paths(paths):
    return pd.concat([gpd.read_file(path) for path in paths])

paths = [os.path.join(data_path, "S_FLD_HAZ_AR.shp") for data_path in eastshore_data_paths]
eastshore_fhl = read_paths(paths)
eastshore_fhl.head()


,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,LEN_UNIT,VELOCITY,VEL_UNIT,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,geometry
0,51001C,1.1.1.0,51001C_81,NP,AE,None,T,8.0,NAVD88,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.33893 37.88938, -75.33921 37.889..."
1,51001C,1.1.1.0,51001C_82,NP,AE,None,T,8.0,NAVD88,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.31579 37.98302, -75.31628 37.982..."
2,51001C,1.1.1.0,51001C_83,NP,OPEN WATER,None,F,-9999.0,None,-9999.0,None,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.30632 38.00647, -75.30685 38.006..."
3,51001C,1.1.1.0,51001C_84,NP,AE,None,T,5.0,NAVD88,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.38663 38.01437, -75.38526 38.014..."
4,51001C,1.1.1.0,51001C_85,NP,AE,None,T,4.0,NAVD88,-9999.0,Feet,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.38742 38.01430, -75.38663 38.014..."


In [10]:
# Create new floodzone: FLD_ZONE = A or AE is 1per, ZONE_SUBTY = 0.2 PCT ANNUAL CHANCE FLOOD HAZARD or AREA WITH REDUCED FLOOD RISK DUE TO LEVEE is 2per
flood_def = [
    (eastshore_fhl['FLD_ZONE'] == 'A') | (eastshore_fhl['FLD_ZONE'] == 'AE'),
    (eastshore_fhl['ZONE_SUBTY'] == '0.2 PCT ANNUAL CHANCE FLOOD HAZARD') | (eastshore_fhl['ZONE_SUBTY'] == 'AREA WITH REDUCED FLOOD RISK DUE TO LEVEE')
]
values = ['1per', '2per']
eastshore_fhl['floodzones'] = np.select(flood_def, values)
eastshore_fhl.head()

,DFIRM_ID,VERSION_ID,FLD_AR_ID,STUDY_TYP,FLD_ZONE,ZONE_SUBTY,SFHA_TF,STATIC_BFE,V_DATUM,DEPTH,...,VELOCITY,VEL_UNIT,AR_REVERT,AR_SUBTRV,BFE_REVERT,DEP_REVERT,DUAL_ZONE,SOURCE_CIT,geometry,floodzones
0,51001C,1.1.1.0,51001C_81,NP,AE,None,T,8.0,NAVD88,-9999.0,...,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.33893 37.88938, -75.33921 37.889...",1per
1,51001C,1.1.1.0,51001C_82,NP,AE,None,T,8.0,NAVD88,-9999.0,...,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.31579 37.98302, -75.31628 37.982...",1per
2,51001C,1.1.1.0,51001C_83,NP,OPEN WATER,None,F,-9999.0,None,-9999.0,...,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.30632 38.00647, -75.30685 38.006...",0
3,51001C,1.1.1.0,51001C_84,NP,AE,None,T,5.0,NAVD88,-9999.0,...,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.38663 38.01437, -75.38526 38.014...",1per
4,51001C,1.1.1.0,51001C_85,NP,AE,None,T,4.0,NAVD88,-9999.0,...,-9999.0,None,None,None,-9999.0,-9999.0,None,51001C_STUDY2,"POLYGON ((-75.38742 38.01430, -75.38663 38.014...",1per


In [11]:
def get_area(a):
    # get area of shp in km^2
    return round(a.geometry.to_crs("EPSG:3395").map(lambda p: p.area / 10**6).iloc[0], 6)

def get_intersection_area(a, b):
    """
    Computes the intersection of a and b and returns the intersection area and size of the intersected area
    as a percentage of the total area of a.
    
    Parameters
    ----------
    a : geopandas GeoDataFrame, required
    b : geopandas GeoDataFrame, required
    """
    intersection = gpd.overlay(a, b, how='intersection')
    if len(intersection) == 0:
        return {'area': 0, 'percent': 0}
    
    # get area of intersection in km^2
    intersect_area = get_area(intersection)
    
    # get area of original shp in km^2
    shp_area = get_area(a)
    
    # compute percentage of intersection of shp
    percentage = (intersect_area / shp_area) * 100
    
    return {'area': intersect_area, 'percent': percentage}


In [12]:
def get_perc_fld_zone_in_region(fhl, sp, logging = False):
    """
    Given National Flood Hazard Layer data (fhl) and spatial boundaries (sp) for counties, tracts, block groups, and blocks,
    this method computes the intersected area between the flood zone and specific geographic area to get an estimate of how much
    of that region is in what flood zone. 
    
    Parameters
    ----------
    fhl : GeoPandas GeoDataFrame, required
        National Flood Hazard Layer data.
        
    sp : dict, required
        Python dictionary containing spatial boundaries for counties, tracts, block groups, and blocks.
        Each spatial unit is a key for a GeoPandas GeoDataFrame with the spatial data.
        
    logging : bool, optional (default is False)
        Whether or not print statements should be executed for more information during runtime.
        
    Output
    ------
    GeoPandas GeoDataFrame
        Each row represents one flood zone type for a geographic region containing the information for how much of 
        that specific region was intersected by that flood zone type.
        
    """
    fld_zones = fhl.floodzones.value_counts().index.tolist()
    area_df = pd.DataFrame()
    for zone in fld_zones:
        if logging:
            print('Starting Zone: {zone}'.format(zone=zone))
        zone_df = fhl[fhl.floodzones == zone].dissolve()
        for su in ['counties', 'tracts', 'blkgps', 'blocks']:
            if logging:
                print('Starting Spatial Unit: {su}'.format(su = su))
                
            area_data = {
                'GEOID': [],
                'zone': [],
                'spatial_unit': [],
                'area': [],
                'perc': []
            }
            
            region = sp[su]
            
            if su == 'tracts':
                # get counties that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "counties") & (area_df.area > 0)]
                intersected_counties = sp['counties'][sp['counties'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only tracts within counties that have been intersected, reduces exploration
                region = region[region.COUNTYFP.isin(intersected_counties.COUNTYFP.values)]
            
            elif su == 'blkgps':
                # get tracts that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "tracts") & (area_df.area > 0)]
                intersected_tracts = sp['tracts'][sp['tracts'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only blkgps within tracts that have been intersected, reduces exploration
                region = region[region.TRACTCE.isin(intersected_tracts.TRACTCE.values)]
                
            elif su == 'blocks':
                # get tracts that were intersected by this flood zone
                intersected_areas = area_df[(area_df.zone == zone) & (area_df.spatial_unit == "blkgps") & (area_df.area > 0)]
                intersected_blkgps = sp['blkgps'][sp['blkgps'].GEOID.isin(intersected_areas.GEOID.values)]
                # filter region to only blocks that have been intersected, reduces exploration
                region = region[region.BLKGRPCE.isin(intersected_blkgps.BLKGRPCE.values)]
                
            for i in tqdm(range(len(region))):
                shp = region.iloc[[i]]
                stats = get_intersection_area(shp, zone_df)
                
                # append data to area data
                area_data['GEOID'].append(shp.GEOID.iloc[0])
                area_data['zone'].append(zone)
                area_data['spatial_unit'].append(su)
                area_data['area'].append(stats['area'])
                area_data['perc'].append(stats['percent'])
                
            area_df = pd.concat([area_df, pd.DataFrame(area_data)])
            
    return area_df

In [13]:
eastshore_area_df = get_perc_fld_zone_in_region(eastshore_fhl, eastshore_sp, logging = True)

Starting Zone: 2per
Starting Spatial Unit: counties


100%|██████████| 2/2 [00:05<00:00,  2.97s/it]


Starting Spatial Unit: tracts


100%|██████████| 16/16 [00:09<00:00,  1.62it/s]


Starting Spatial Unit: blkgps


...

100%|██████████| 16/16 [00:10<00:00,  1.48it/s]


Starting Spatial Unit: blkgps


100%|██████████| 46/46 [00:24<00:00,  1.89it/s]


Starting Spatial Unit: blocks


100%|██████████| 6469/6469 [55:52<00:00,  1.93it/s]  


In [14]:
pwd

'/Users/micheleclaibourn/Box Sync/mpc/dataForDemocracy/summer-sandbox/eastern_shore_collection'

In [15]:
for key in ['counties', 'tracts', 'blkgps', 'blocks']:
    eastshore_area_df[eastshore_area_df.spatial_unit == key].to_csv('dataraw/nfhl_eastern_{key}.csv'.format(key = key))